## 🧭 Notebook 5 — Robustness and Policy Counterfactuals

### 🟩 5.0 Setup and Data Loading

In [1]:
# ============================================================
# 🧩 Notebook 5 — Robustness and Policy Counterfactuals
# ============================================================

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from linearmodels import PanelOLS
import statsmodels.api as sm
import joblib
import shap
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Display settings
sns.set_style("whitegrid")
plt.rcParams.update({"figure.dpi":120, "axes.titlesize":13, "axes.labelsize":11})

# --- Paths ---
DATA_DIR = Path("../outputs/data")
MODEL_DIR = Path("../outputs/models")

# --- Load panel data and predictions ---
panel = pd.read_parquet(DATA_DIR / "panel_final.parquet")
panel["year"] = panel["year"].astype(int)
panel["region"] = panel["region"].astype(str)

print(f"✅ Loaded panel with {len(panel):,} rows and {panel.shape[1]} columns")

# --- Load trained models ---
xgb_model = joblib.load(MODEL_DIR / "xgb_model.pkl")
lgbm_model = joblib.load(MODEL_DIR / "lgbm_model.pkl")

/Users/golibsanaev/Library/CloudStorage/Dropbox/GitHub_gsanaev/forecasting-explaining-hotel-demand-in-eu/.venv-lstm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: '../outputs/data/panel_final.parquet'

### 🟩 5.1 Placebo & Pre-Trend Tests

In [ ]:
# ============================================================
# 🔍 5.1 Placebo & Pre-Trend Tests
# ============================================================

# Define fake treatment year (placebo)
placebo_year = 2018
panel["treated"] = panel["region"].isin(["IT","ES","FR"]).astype(int)
panel["post_placebo"] = (panel["year"] >= placebo_year).astype(int)
panel["did_placebo"] = panel["treated"] * panel["post_placebo"]

# Run placebo DiD
y = panel["nights_spent"]
X = sm.add_constant(panel[["treated", "post_placebo", "did_placebo"]])
mod_placebo = PanelOLS(y, X, entity_effects=True, time_effects=True)
res_placebo = mod_placebo.fit(cov_type="clustered", clusters=panel["region"])
print(res_placebo.summary)

### 🟩 5.2 Alternative Control Groups

In [ ]:
# ============================================================
# 🧪 5.2 Alternative Control Groups
# ============================================================

# Drop outliers / special cases
alt = panel[~panel["region"].isin(["MT", "CY", "LU"])].copy()

alt["did"] = alt["treated"] * (alt["year"] >= 2020)

y = alt["nights_spent"]
X = sm.add_constant(alt[["treated", "did"]])

mod_alt = PanelOLS(y, X, entity_effects=True, time_effects=True)
res_alt = mod_alt.fit(cov_type="clustered", clusters=alt["region"])
print(res_alt.summary)

### 🟩 5.3 Sensitivity and Specification Checks

In [ ]:
# ============================================================
# ⚙️ 5.3 Sensitivity & Specification Checks
# ============================================================

from sklearn.metrics import mean_squared_error, mean_absolute_error
def rmse(y, yhat): return np.sqrt(mean_squared_error(y, yhat))

results = []
for depth in [4,6,8]:
    xgb_temp = XGBRegressor(
        n_estimators=400, learning_rate=0.05, max_depth=depth,
        subsample=0.9, colsample_bytree=0.9, random_state=42
    )
    X = panel[[c for c in panel.columns if c.endswith(("_lag1","_lag2","_lag3","_mom"))]]
    y = panel["nights_spent"]
    xgb_temp.fit(X, y)
    panel[f"yhat_depth{depth}"] = xgb_temp.predict(X)
    results.append({
        "max_depth": depth,
        "RMSE": rmse(y, panel[f"yhat_depth{depth}"]),
        "MAE": mean_absolute_error(y, panel[f"yhat_depth{depth}"])
    })

pd.DataFrame(results)

### 🟩 5.4 Synthetic Control (Optional)

In [ ]:
# ============================================================
# 🧮 5.4 Synthetic Control — Example
# ============================================================
# (Simplified demo: construct synthetic control for Italy)

donor = panel[~panel["region"].isin(["IT"])].pivot(index="year", columns="region", values="nights_spent")
target = panel[panel["region"]=="IT"].set_index("year")["nights_spent"]

weights = donor.loc[donor.index<2020].corrwith(target[donor.index<2020])
weights = np.maximum(weights,0)
weights /= weights.sum()

synth = (donor * weights).sum(axis=1)
plt.figure(figsize=(7,4))
plt.plot(target, label="Italy (Actual)")
plt.plot(synth, label="Synthetic Italy", linestyle="--")
plt.axvline(2020, color="red", linestyle="--")
plt.title("Synthetic Control vs Actual: Italy")
plt.legend()
plt.show()

### 🟩 5.5 Policy Scenario Forecasting

In [ ]:
# ============================================================
# 🌍 5.5 Policy Scenario Forecasting
# ============================================================

future = panel[panel["year"]>=2020].copy()
X_future = future[[c for c in panel.columns if c.endswith(("_lag1","_lag2","_lag3","_mom"))]]

future["yhat_xgb"] = xgb_model.predict(X_future)
future["yhat_lgbm"] = lgbm_model.predict(X_future)
future["counterfactual"] = future["yhat_xgb"] * 1.1  # simulate +10% tourism policy boost

plt.figure(figsize=(8,4))
for reg in ["IT","ES","FR"]:
    subset = future[future["region"]==reg]
    plt.plot(subset["year"], subset["yhat_xgb"], label=f"{reg} – baseline")
    plt.plot(subset["year"], subset["counterfactual"], linestyle="--", label=f"{reg} – +10% policy")

plt.axvline(2020, color="red", linestyle="--")
plt.title("Policy Counterfactual: Tourism Boost Scenario")
plt.ylabel("Predicted Nights Spent")
plt.legend()
plt.show()

### 🟩 5.6 Final Summary & Export

In [ ]:
# ============================================================
# 📦 5.6 Final Summary & Export
# ============================================================

summary = {
    "Placebo DiD": res_placebo.params["did_placebo"],
    "Alternative Controls DiD": res_alt.params["did"],
    "Best XGB Depth": min(results, key=lambda r: r["RMSE"])["max_depth"]
}
pd.DataFrame([summary]).T.rename(columns={0: "Value"}).to_csv("../outputs/final_summary.csv")
print("✅ Final summary exported.")